<a href="https://colab.research.google.com/github/jiminAn/2023_SUMMER_SAMSUNG_DL/blob/main/%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C/Day4/Day_4_3_RNN_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 4.3 RNN Basic
### 실습 목표
- RNN 모델의 입력 데이터 형태를 살펴보고 모델을 정의해보자

### Contents
1. RNN 모델의 입력 데이터 형태
2. RNN 모델 정의
  1. vanila RNN
  2. LSTM


## 0. Setting
- DL 관련 library 호출
- GPU 설정

In [ ]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


## 1. RNN 모델의 입력 데이터 형태
### 예시
- "hello"를 one-hot encoding을 사용하여 벡터로 치환해봅시다


In [ ]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [ ]:
input_data_np = np.array([[h, e, l, l, o]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])

- 앞에서 선언한 벡터를 이용하여 추가로, "eolll", "lleel"를 벡터로 치환해봅시다

In [ ]:
# transform as torch tensor
input_data_np = np.array([[h, e, l, l, o],
                          [e, o, l, l, l],
                          [l, l, e, e, l]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

In [ ]:
# (batch size, seq length(time step), input_size)
input_data.size()

torch.Size([3, 5, 4])

## 1. RNN 계열 모델 정의
1. Vanila RNN
2. LSTM




### 2.1 Vanila RNN
<img src = "https://drive.google.com/uc?id=1jT028MtyD7Gg4yEFsDJkwaRnK9bJx1u2" height=350 width=500>

#### `torch.nn.RNN(input_size, hidden_size, batch_first)`
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
- `input_size`: 입력해 주는 특성 값의 개수
- `hidden_size`: hidden state의 사이즈 지정. 보통 arbitrary 함.
- `batch_first`: 입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정.
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [ ]:
# declare dimension
input_size = 4
hidden_size = 3

# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  # 처음에 batch_size or seq_length

In [ ]:
# check output
outputs, _status = rnn(input_data)

print("--------------모든 time steps의 hidden states-----------------")
print("outputs:\n", outputs)
print("outputs size:",outputs.size()) # (5번의 time step동안 3차원은 은닉 상태가 출력됨)*단어 3개
# shape : (3, 5, 3)
# (batch_size, seq_length, hidden_dim)
print("--------------마지막 시점의 hidden state-----------------")
print("hidden state:\n", _status) # 마지막 시점의 hidden state
print("hidden state size:",_status.size())
# shape: (1, 3, 3)
# (num_layers, batch_size, hidden_dim)

--------------모든 time steps의 hidden states-----------------
outputs:
 tensor([[[ 0.1875, -0.4714, -0.0445],
         [ 0.4501, -0.1990, -0.5874],
         [ 0.0320, -0.6834, -0.1961],
         [-0.1507, -0.6082,  0.0226],
         [ 0.4725, -0.5835, -0.5305]],

        [[ 0.4068, -0.2182, -0.5961],
         [ 0.6644, -0.6627, -0.7026],
         [ 0.1136, -0.6730, -0.2026],
         [-0.1383, -0.6211,  0.0037],
         [-0.2434, -0.5987,  0.0876]],

        [[-0.2408, -0.6675, -0.0191],
         [-0.2470, -0.5769,  0.1093],
         [ 0.3586, -0.0955, -0.4956],
         [ 0.5793, -0.2456, -0.6877],
         [ 0.0864, -0.6919, -0.2338]]], grad_fn=<TransposeBackward1>)
outputs size: torch.Size([3, 5, 3])
--------------마지막 시점의 hidden state-----------------
hidden state:
 tensor([[[ 0.4725, -0.5835, -0.5305],
         [-0.2434, -0.5987,  0.0876],
         [ 0.0864, -0.6919, -0.2338]]], grad_fn=<StackBackward0>)
hidden state size: torch.Size([1, 3, 3])


## 2.2 LSTM
<img src = "https://drive.google.com/uc?id=1sVemx9k_QiN4pRAH_ZCE-D6mGaW4hlar" height=350 width=500>


#### `torch.nn.LSTM(input_size, hidden_size, batch_first)`
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
- `input_size`: 입력해 주는 특성 값의 개수
- `hidden_size`: hidden state의 사이즈 지정. 보통 arbitrary 함.
- `batch_first`: 입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정.
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [ ]:
lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
outputs, _status = lstm(input_data)
print(outputs)
print(outputs.size())
print()

print(_status[0])
print(_status[0].shape)

tensor([[[ 7.7160e-02,  4.8831e-02, -1.9702e-01],
         [ 2.3895e-01, -7.7501e-02, -3.6145e-01],
         [ 2.8074e-01, -3.8985e-02, -4.9372e-01],
         [ 3.1611e-01, -2.2627e-02, -5.2396e-01],
         [ 5.0144e-01,  2.8663e-02, -4.3278e-01]],

        [[ 1.9160e-01, -1.0361e-01, -2.4555e-01],
         [ 3.9922e-01, -4.2685e-04, -3.0189e-01],
         [ 3.2583e-01, -4.2673e-02, -4.2808e-01],
         [ 3.4120e-01, -3.5685e-02, -4.9169e-01],
         [ 3.5151e-01, -2.8928e-02, -5.2391e-01]],

        [[ 1.3010e-01, -3.0247e-02, -2.8644e-01],
         [ 2.2089e-01, -2.2524e-02, -4.2176e-01],
         [ 3.2510e-01, -8.9915e-02, -4.3805e-01],
         [ 3.6905e-01, -1.1317e-01, -4.8559e-01],
         [ 3.5704e-01, -5.3226e-02, -5.8023e-01]]],
       grad_fn=<TransposeBackward0>)
torch.Size([3, 5, 3])

tensor([[[ 0.5014,  0.0287, -0.4328],
         [ 0.3515, -0.0289, -0.5239],
         [ 0.3570, -0.0532, -0.5802]]], grad_fn=<StackBackward0>)
torch.Size([1, 3, 3])
